## Titanic 乘客生存预测（决策树）

### 训练和测试数据见 data 文件夹中的 Titanic_train.csv 和 Titanic_test.csv

### 字段
* PassengerId 乘客编号
* Survived 是否幸存
* Pclass 船票等级
* Name 乘客姓名
* Sex 乘客性别
* Age 乘客年龄
* SibSp 亲戚数量（兄妹、配偶数）
* Parch 亲戚数量（父母、子女数）
* Ticket 船票号码
* Fare 船票价格
* Cabin 船舱
* Embarked 登陆港口

### 流程
1. **准备阶段**：数据探索 -> 数据清洗 -> 特征选择。
2. **分类阶段**：决策树模型 -> 模型评估&预测 -> 决策树可视化

### 模块1：数据探索
* info()：了解数据表基本情况，行数、列数、每列的数据类型、数据完整度 等；
* describe()：了解数据表的统计情况，总数、平均值、标准差、最大/最小值 等；
* describe(include=[ 'O' ])：查看字符串类型（非数字）的整体情况；
* head()：查看前几行的数据（默认为前 5 行）
* tail()：查看后几行的数据（默认为后 5 行）

In [1]:
import pandas as pd

# 数据加载
train_data = pd.read_csv('./data/Titanic_train.csv')
test_data = pd.read_csv('./data/Titanic_test.csv')

# 数据探索
print(train_data.info())
print("-"*30)

print(train_data.describe())
print("-"*30)

print(train_data.describe(include=['O']))
print("-"*30)

print(train_data.head())
print("-"*30)

print(train_data.tail())
print("-"*30)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None
------------------------------
       PassengerId    Survived      Pclass         Age       SibSp       Parch        Fare
count   891.000000  891.000000  891.000000  714.000000  891.000000  891.000000  891.000000
mean    446.000000    0.383838    2.308642   29.699118    0.523008    0.381594   32.204208
std     257.353842    0.486592    0.836071   14.526497    1.102743    0.806057   49.693429
min       1.000

### 模块2：数据清洗
**Age、Fare、Cabin、Embarked 均有缺失值**
1. Age：年龄，数值型，用平均值补齐
2. Fare：船票价格，数值型，用平均值补齐（测试集中有缺失值）
3. Cabin：船舱，缺失值过多，无法补齐
4. Embarked：登陆港口，少量缺失值，有 3 个登陆港口，用占比最多的填充缺失值

In [2]:
# 使用平均年龄填充年龄中的缺失值
train_data["Age"].fillna(train_data["Age"].mean(), inplace=True)
test_data["Age"].fillna(test_data["Age"].mean(), inplace=True)

# 使用票价的均值填充票价中的缺失值
train_data["Fare"].fillna(train_data["Fare"].mean(), inplace=True)
test_data["Fare"].fillna(test_data["Fare"].mean(), inplace=True)

In [3]:
print(train_data["Embarked"].value_counts())

S    644
C    168
Q     77
Name: Embarked, dtype: int64


In [4]:
# 使用登陆最多的港口来填充缺失值
train_data["Embarked"].fillna("S", inplace=True)
test_data["Embarked"].fillna("S", inplace=True)

### 模块3：特征选择
* PassengerId、Name 对分类无特别作用，放弃
* Cabin 缺失值太多，放弃
* Ticket 船票号码，杂乱无规律，无意义，放弃
* Pclass、Sex、Age、SibSp、Parch、Fare：乘客的船票等级、性别、年龄、亲戚数量、船票价格，选为特征

In [13]:
# 特征选择
features = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]

train_features = train_data[features]
train_labels = train_data["Survived"]

test_features = test_data[features]

***将特征值中的字符串转成数值类型，便于后续计算***
* Sex 字段：Female 和 Male 可用数值 0 和 1 表示
* Embarked 字段：S C Q 三种可能，同理转换为数值
* 使用 sklearn 的 DictVertorizer 类
* fit_transform()：将特征向量转化为特征值矩阵

In [14]:
from sklearn.feature_extraction import DictVectorizer

dvec = DictVectorizer(sparse=False)
train_features = dvec.fit_transform(train_features.to_dict(orient="records"))

In [15]:
print(dvec.feature_names_)

['Age', 'Embarked=C', 'Embarked=Q', 'Embarked=S', 'Fare', 'Parch', 'Pclass', 'Sex=female', 'Sex=male', 'SibSp']


***train_features 特征矩阵包括 10 个特征值（列），891 个样本（行），即 891 * 10 的特征矩阵***

### 模块4：决策树模型
**ID3算法，即指定 DecisionTreeClassifier 的 criterion="entropy"**

In [16]:
from sklearn.tree import DecisionTreeClassifier

# 构造 ID3 决策树
clf = DecisionTreeClassifier(criterion="entropy")
# 决策树训练
clf.fit(train_features, train_labels)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

### 模块5：模型预测&评估
**得到测试集的特征矩阵，用训练好的决策树进行预测，得到预测结果**

In [17]:
test_features = dvec.transform(test_features.to_dict(orient="records"))

# 决策树预测
pred_labels = clf.predict(test_features)

***因无测试集实际结果，所以需采用 K 折交叉验证的方式***
1. 将数据集平均分割成 k 个等份
2. 使用 1 份作为测试数据，其余作为训练数据
3. 计算测试准确率
4. 使用不同的测试集，重复 2 3 步骤
5. sklearn 的 model_selection 中的 cross_val_score()，cv 指定将原始数据划分为多少份，一般取 10

In [18]:
import numpy as np
from sklearn.model_selection import cross_val_score

# 使用 k 折交叉验证，统计决策树准确率
print("cross_val_score 准确率为 %.4lf" % np.mean(cross_val_score(clf, train_features, train_labels, cv=10)))

cross_val_score 准确率为 0.7824


### 模块6：决策树可视化
**通过 Graphviz 可视化**

In [20]:
from sklearn import tree
import graphviz

dot_data = tree.export_graphviz(clf, out_file=None)
graph = graphviz.Source(dot_data)
graph.view()

'Source.gv.pdf'